## 2.6. Document Validation

By design, MongoEngine strictly validates the documents right before they are inserted in MongoDB and makes sure they are consistent with the fields defined in your models.

设计上，**当你保存文档之前**，MongoEngine严格验证他们确保符合你在模型中定义的字段。

MongoEngine makes the assumption that the documents that exists in the DB are compliant with the schema. This means that Mongoengine will not validate a document when an object is loaded from the DB into an instance of your model but this operation may fail under some circumstances (e.g. if there is a field in the document fetched from the database that is not defined in your model).

MongoEngine 假设已存在数据库中的文档是符合设计模式的。因此，使用你创建的MongoEngine模型读取数据库中的文档并加载为一个实例时，在某些情况下存在失败(这里的失败指不能准确匹配字段，而不是直接不能读取并报错)的可能。例如，一个已存在的文档缺少你模型中定义的字段，这也意味着，产生这种情况在读取时不会报错。

### 建立的模型同数据库中字段不一致情况测试

In [1]:
from mongoengine import connect, Document, StringField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)


MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [11]:
class Seller(Document):
    name = StringField()
    # phone = StringField(regex="^\d+")
    phone2 = StringField(regex="^\d+$")
    meta = {"db_alias": TEST_DB1}

In [3]:
Seller.objects().limit(2).to_json(ensure_ascii=False)

'[{"_id": {"$oid": "64086a70248d96a15f6ff2e9"}, "name": "当当书店", "phone": "1288753"}, {"_id": {"$oid": "64087b3d248d96a15f6ff2ec"}, "name": "恩悌网络有限公司", "phone": "15836709368"}]'

In [4]:
Seller.objects().count()

325

### 2.6.1. Built-in validation(内置验证)

Mongoengine provides different fields that encapsulate the corresponding validation out of the box. Validation runs when calling `.validate()` or `.save()`

Mongoengine提供了封装了相应开箱即用验证的不同字段。当运行 `.validate()` or `.save()`函数时执行验证。

```python
from mongoengine import Document, EmailField, IntField

class User(Document):
    email = EmailField()
    age = IntField(min_value=0, max_value=99)

user = User(email='invalid@', age=24)
user.validate()     # raises ValidationError (Invalid email address: ['email'])
user.save()         # raises ValidationError (Invalid email address: ['email'])

user2 = User(email='john.doe@garbage.com', age=1000)
user2.save()        # raises ValidationError (Integer value is too large: ['age'])
```

In [ ]:
Seller(phone2="123m").validate()

### 2.6.2. Custom validation(定制验证)

The following feature can be used to customize the validation:  
有以下三种方法可以定制验证, 字段(`Field`)的 `validation`参数, 文档(`Document`)的 `clean` 方法，改写 `Field`类的`validate`函数


#### `Field` `validation` parameter  
字段(`Field`)的 `validation`参数，可以被用来定制验证函数。

```python
def not_john_doe(name):
    if name == 'John Doe':
        raise ValidationError("John Doe is not a valid name")

class Person(Document):
    full_name = StringField(validation=not_john_doe)

Person(full_name='Billy Doe').save()
Person(full_name='John Doe').save()  # raises ValidationError (John Doe is not a valid name)
```


#### Document `clean` method

This method is called as part of `save()` and should be used to provide custom model validation and/or to modify some of the field values prior to validation. For instance, you could use it to automatically provide a value for a field, or to do validation that requires access to more than a single field.

文档的 `clean` 也可以被用来验证字段是否合法。`clean` 实际上是 `save()` 的一部分，可以理解为 `save()` 执行前先执行 `clean`。  
可以使用这种方法编写验证逻辑，或多字段间的相互验证。

```python
class Essay(Document):
    status = StringField(choices=('Published', 'Draft'), required=True)
    pub_date = DateTimeField()

    def clean(self):
        # Validate that only published essays have a `pub_date`
        if self.status == 'Draft' and self.pub_date is not None:
            raise ValidationError('Draft entries should not have a publication date.')
        # Set the pub_date for published items if not set.
        if self.status == 'Published' and self.pub_date is None:
            self.pub_date = datetime.now()
```

> Cleaning is only called if validation is turned on and when calling save().

`clean` 只有在验证开启并调用 `save`方法时执行。

#### Adding custom Field classes  
改写 `Field`类的`validate`函数

We recommend as much as possible to use fields provided by MongoEngine. However, it is also possible to subclass a Field and encapsulate some validation by overriding the validate method

```python
class AgeField(IntField):

    def validate(self, value):
        super(AgeField, self).validate(value)     # let IntField.validate run first
        if value == 60:
            self.error('60 is not allowed')

class Person(Document):
    age = AgeField(min_value=0, max_value=99)

Person(age=20).save()   # passes
Person(age=1000).save() # raises ValidationError (Integer value is too large: ['age'])
Person(age=60).save()   # raises ValidationError (Person:None) (60 is not allowed: ['age'])
```

注意改写时，需要继承 原始 `validate` 的内容,即改写前执行 `supper`方法。

### 2.6.3. Skipping validation

Although discouraged as it allows to violate fields constraints, if for some reason you need to disable the validation and cleaning of a document when you call save(), you can use .save(validate=False).

```python
class Person(Document):
    age = IntField(max_value=100)

Person(age=1000).save()    # raises ValidationError (Integer value is too large)

Person(age=1000).save(validate=False)
person = Person.objects.first()
assert person.age == 1000
```
